<a href="https://colab.research.google.com/github/Amarmurun0212/Diver/blob/main/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Problem 1] Execution of machine translation and code reading

In [5]:
pwd

'/content'

In [6]:
%cd /content/

/content


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!wget http://www.manythings.org/anki/fra-eng.zip

--2023-05-30 08:35:47--  http://www.manythings.org/anki/fra-eng.zip
Resolving www.manythings.org (www.manythings.org)... 173.254.30.110
Connecting to www.manythings.org (www.manythings.org)|173.254.30.110|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7420323 (7.1M) [application/zip]
Saving to: ‘fra-eng.zip’

fra-eng.zip         100%[===================>]   7.08M  43.9MB/s    in 0.2s    

2023-05-30 08:35:48 (43.9 MB/s) - ‘fra-eng.zip’ saved [7420323/7420323]



In [9]:
!unzip fra-eng.zip

Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 


In [12]:
lines[0]

'Go.\tVa !\tCC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)'

In [21]:
# ライブラリのインポート
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

#各種パラメータの設定
batch_size = 64  
epochs = 100  
latent_dim = 256 
num_samples = 10000 
data_path = 'drive/My Drive/Diver code/seq2seq/fra.txt'


input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

#言語データを開いて、変数に格納
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
#1行ずつ呼び出して、input, targetのリストに分ける。
#更に行を単語ごとに分割して、input_characters, target_charactersへ格納する。
# \tは始まりの合図、\nは終わりの合図
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

#各種リストの並び替えや、変数の格納
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#表示
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#input,targetのトークン化(それぞれの単語にidを割り当てる。)
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#入れ物を用意
#encoderの入力データ,shape(入力データ数, 入力データの最大単語数, 入力データの単語数)
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32') 
    
#decoderの入力データ shape(入力データ数, 入力データの最大単語数, 入力データの単語数)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32') 
    
#decoderの出力データ shape(入力データ数, 入力データの最大単語数, 入力データの単語数)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#入れ物にワンホットラベル化して入れる
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

#モデルビルディング(encoder)
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

#モデルビルディング(decoder)
decoder_inputs = Input(shape=(None, num_decoder_tokens))
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states) #initial_stateでencoder_statesを受け取る。
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

#encoder,decoderを混ぜてモデル作成。
#encoder, decoderのインプットが入力データ、decoder_outputsが教師データ
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
model.save('s2s.h5')

#推論のモデルを作成
#encoderのinputが入力データ、encoderのstatesを教師データとする。
encoder_model = Model(encoder_inputs, encoder_states)

#decoderはencoderのpredictをstatesとして、target_sequenceをinputとしてpredectするようなモデルを作る。
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs) #decoder_inputを入力データ、initial_stateにdecoder_states_inputを渡す。
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

#予測値はワンホットベクトル化されているので、言語に戻す。
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


#predictする関数の作成
def decode_sequence(input_seq):
    #encoderで予測
    states_value = encoder_model.predict(input_seq)
    
    #decoderに入力するベクトルの作成(最初はスタートindex)
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.
    
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        
        #decoderで予測(inputは\t or 自分の予測値, statesはencoder_predict or decoder_states)
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)
        
        #softmaxからの変換や、idから言語への変換等
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char
        
        #終わりの合図or限界が来たらfor文停止、
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True
            
        #encoderの予測値をtarget_seqに入れ、statesも更新して、while文をまた回す。
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.
        states_value = [h, c]

    return decoded_sentence


#100個のサンプル文章を推論している。
for seq_index in range(100):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

ValueError: ignored